In [ ]:
install.packages("BiocManager")
install.packages("forcats")
install.packages("stringr")
install.packages("ggplot2")
install.packages("ggrepel")
install.packages("readr")
install.packages("tidyr")
install.packages("survminer")
BiocManager::install("GEOquery")
BiocManager::install("limma")
BiocManager::install("pheatmap")
BiocManager::install("org.Hs.eg.db")

In [ ]:
library(GEOquery)
id1 <- "GSE65194"
id2 <- "GSE43358"
id3 <- "GSE76275"
id4 <- "GSE45827"
id5 <- "GSE7904"
id6 <- "GSE95700"
id7 <- "GSE58812"
id8 <- "GSE48390"

In [ ]:
gse1 <- getGEO(id1)

In [ ]:
gse2 <- getGEO(id2)

In [ ]:
gse3 <- getGEO(id3)

In [ ]:
gse4 <- getGEO(id4)

In [ ]:
gse5 <- getGEO(id5)

In [ ]:
gse6 <- getGEO(id6)

In [ ]:
gse7 <- getGEO(id7)

In [ ]:
gse8 <- getGEO(id8)

In [ ]:
gse <- rbind(gse1, gse2, gse3, gse4, gse5, gse6, gse7, gse8)
dim(gse)

In [ ]:
gse_0 <- gse[[1]]
gse_2 <- gse[[2]]
gse_3 <- gse[[3]]
gse_4 <- gse[[4]]
gse_5 <- gse[[5]]
gse_6 <- gse[[6]]
gse_7 <- gse[[7]]
gse_8 <- gse[[8]]
dim(gse_8)

In [ ]:
pData(gse_8)

In [14]:
gse_1 <- combine(gse_0, gse_2, gse_3, gse_4, gse_5, gse_6, gse_7, gse_8)
gse_1 <- na.omit(gse_1)

In [ ]:
dim(gse_1)

In [16]:
options(repr.matrix.max.cols=150, repr.matrix.max.rows=1000)

In [ ]:
exprsData <- exprs(gse_1)
exprsData <- na.omit(exprsData)
exprsData
dim(exprsData)

In [ ]:
library(dplyr)

In [ ]:
sampleInfo <- pData(gse_0)
sampleInfo <- transform(sampleInfo,bc_type=ifelse(characteristics_ch1=="sample_group: TNBC","TNBC","non-TNBC"))
#sampleInfo <- transform(sampleInfo,non_TNBC=ifelse(characteristics_ch1=="sample_group: TNBC",0,1))
sampleInfo <- select(sampleInfo, geo_accession, bc_type)

sample2 <- pData(gse_2)
sample2 <- transform(sample2,bc_type=ifelse((characteristics_ch1=="er: 0")&
                                        (characteristics_ch1.1=="pgr: 0")&
                                        (characteristics_ch1.2=="her2: 0"),"TNBC","non-TNBC"))
# sample2 <- transform(sample2,non_TNBC=ifelse((characteristics_ch1=="er: 0")&
#                                         (characteristics_ch1.1=="pgr: 0")&
#                                         (characteristics_ch1.2=="her2: 0"),0,1))
sample2 <- select(sample2, geo_accession, bc_type)

sample3 <- pData(gse_3)
sample3 <- cbind(sample3, bc_type = ifelse(grepl(" not TN",sample3$'set:ch1'),"non-TNBC","TNBC"))
# sample3 <- cbind(sample3, non_TNBC = 0)
sample3 <- select(sample3, geo_accession, bc_type)

sample4 <- pData(gse_4)
sample4 <- cbind(sample4, bc_type = "non-TNBC")
sample4 <- select(sample4, geo_accession, bc_type)

sample5 <- pData(gse_5)
sample5 <- cbind(sample5, bc_type = "non-TNBC")
sample5 <- select(sample5, geo_accession, bc_type)

sample6 <- pData(gse_6)
sample6 <- cbind(sample6, bc_type = "TNBC")
sample6 <- select(sample6, geo_accession, bc_type)

sample7 <- pData(gse_7)
sample7 <- cbind(sample7, bc_type = "TNBC")
sample7 <- select(sample7, geo_accession, bc_type)

sample8 <- pData(gse_8)
sample8 <- cbind(sample8, bc_type = "non-TNBC")
sample8 <- select(sample8, geo_accession, bc_type)

sampleInfo <- rbind(sampleInfo, sample2, sample3, sample4, sample5, sample6, sample7, sample8)
#sampleInfo <- select(sampleInfo, geo_accession, TNBC)
show(sampleInfo)

In [ ]:
summary(exprsData)

In [21]:
#exprs(gse_1) <- log2(exprs(gse_1))
#boxplot(exprs(gse_1),outline=FALSE)

In [ ]:
library(limma)
design <- model.matrix(~0+sampleInfo$bc_type)
colnames(design) <- c("non_TNBC", "TNBC")
design
dim(design)

In [ ]:
summary(exprsData)

## calculate median expression level
cutoff <- median(exprsData)

## TRUE or FALSE for whether each gene is "expressed" in each sample
is_expressed <- exprsData > cutoff

## Identify genes expressed in more than 2 samples

keep <- rowSums(is_expressed) > 2

## check how many genes are removed / retained.
table(keep)

## subset to just those expressed genes
gse_1 <- gse_1[keep,]
gse_1
dim(gse_1)

In [ ]:
fit <- lmFit(exprsData, design)
fit$coefficients

In [ ]:
contrast1 <- makeContrasts(TNBC - non_TNBC, levels=design)
                           
## define multiple contrasts
## e.g. makeContrasts(Group1 - Group2, Group2 - Group3,....levels=design)
## Healthy - TNBC, Her2 - TNBC, LuminalA - TNBC, LuminalB - TNBC,
fit1 <- contrasts.fit(fit, contrast1)
fit1 <- eBayes(fit1)
topTable(fit1)
table(decideTests(fit1))

results <- topTable(fit1, number=Inf)
results <- tibble::rownames_to_column(results,"ID")

In [26]:
# contrast2 <- makeContrasts(Healthy - TNBC, levels=design)

# fit2 <- contrasts.fit(fit, contrast1)
# fit2 <- eBayes(fit2)
# topTable(fit2)
# #decideTests(fit2)
# table(decideTests(fit2))

# part_results_2 <- topTable(fit2, number=Inf)
# part_results_2 <- tibble::rownames_to_column(part_results_2,"ID")

In [27]:
# library(ggplot2)
# ggplot(full_results,aes(x = logFC, y=B)) + geom_point()

In [ ]:
p_cutoff <- 0.05
fc_cutoff <- 1

final <- filter(results, adj.P.Val < 0.05, abs(logFC) > 1)
#final
library(readr)
write_csv(final, path="final_results.csv")
dim(final)

In [29]:
# library(readr)
# filter(results, adj.P.Val < 0.05, abs(logFC) > 1) %>%
#   write_csv(path="filtered_de_results.csv")

In [30]:
options(repr.matrix.max.cols=200, repr.matrix.max.rows=40000)
# DT2 <- df(filter(exprs(gse_1), ID %in% results$ID))

DT1 <- exprsData

#head(DT1)


d <- DT1
ID <- rownames(d)
rownames(d) <- NULL
data <- cbind(ID,d)
#data

# DT2 <- subset(data, ID %in% results$ID)
# DT2

In [ ]:
DT2 <- subset(data, ID %in% final$ID)
#write_csv(DT2, path="model_input.csv")
dim(DT2)
head(DT2)
class(DT2)

In [ ]:
df <- as.data.frame(DT2)
df2 <- data.frame(t(df[-1]))
colnames(df2) <- df[, 1]
# head(df2)
# dim(df2)

df3 <- cbind(df2, type = sampleInfo$bc_type)

d1 <- df3
geo_ID <- rownames(d1)
rownames(d1) <- NULL
df3 <- cbind(geo_ID,d1)
head(df3)
dim(df3)

write_csv(df3, path="new_input.csv")